In [2]:
import numpy as np, humanfriendly as hf, warnings, sys
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error

In [3]:
def get_error(model, Xtest, ytest):
    y_pred = model.predict(Xtest)
    return np.sqrt(mean_squared_error(ytest, y_pred)),model.__class__.__name__

In [4]:
def see_time(note,start):
    end = time.perf_counter()
    elapsed = end - start
    print (note,hf.format_timespan(elapsed, detailed=True))

In [5]:
def get_cross(model, data, target, groups=10):
    return cross_val_score(model, data, target, cv=groups,scoring='neg_mean_squared_error')

In [6]:
X = np.load('data/X_boston.npy')
y = np.load('data/y_boston.npy')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [8]:
rfr_model = RandomForestRegressor(random_state=0)
rfr_model.fit(X_train, y_train)
rmse, rfr_model_name = get_error(rfr_model, X_test, y_test)
print(rfr_model_name + '(rmse):',rmse)

RandomForestRegressor(rmse): 3.5587794792757004


C:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [9]:
n_est = [100, 500, 1000]
boot = [True, False]
params = {'n_estimators': n_est, 'bootstrap': boot}
grid = GridSearchCV(rfr_model, params, cv=5, n_jobs=-1,verbose=1, refit=False)
start = time.perf_counter()
grid.fit(X_train, y_train)
see_time('training time:',start)
bp = grid.best_params_
print (bp)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


training time: 14 seconds, 502 milliseconds, 243 microseconds and 100 nanoseconds
{'bootstrap': True, 'n_estimators': 100}


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   14.4s finished


In [12]:
rfr_bpmodel = RandomForestRegressor(**bp, random_state=0)
rfr_bpmodel.fit(X_train, y_train)
rmse,rfr_bpmodel_name = get_error(rfr_bpmodel, X_test, y_test)
print(rfr_bpmodel_name + '(rmse):',rmse)

RandomForestRegressor(rmse): 3.37169151536684


In [13]:
start = time.perf_counter()
scores = get_cross(rfr_bpmodel, X, y)
see_time('cross-validation rmse:',start)
rmse = np.sqrt(np.mean(scores) * -1)
print (rmse)

cross-validation rmse: 3 seconds, 52 milliseconds, 985 microseconds and 800 nanoseconds
3.6815463792891623
